# Homework 1
By Quentin Barnes  
CS 344

### Part one: Introspection:
Introspection is the practice of looking inward, towards one’s own thoughts in an attempt to gain insight on how we work.  It is a practice that is as old as Plato and is very useful in the field of psychology.  Because of this, it is useful in modeling for AI, however only to an extent.  It is useful because a lot of the times we can teach the computers to solve the problems the same ways that we do.  This does have some limitations and drawbacks, though, since computers are mechanical in nature.  Sometimes the way that we do things in our heads is very inefficient because we do have limitations.  
  
I think that an interesting area to look at is vision.  As we are teaching computers to actually see, we are doing it in a way that is similar to how we learned to see.  For computers we have people label a bunch of objects and then feed it to programs to let them learn what those objects look like, which is a lot like how humans learn to recognize things.  However, even in this case, they don’t do it exactly the same, and the program can also be fooled a lot easier.  

### Part 2: Traveling Salesman Probelm: 

In [1]:
"""
This module implements local search on a simple sine function variant.  It uses
simulated annealing and hill climbing to do this.  It also implements random restarts
to average out the solutions and find a max over a given number of runs.

@author: Quentin Barnes
(Based on code by kvlinden)
"""
import math
import time
from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange


# class AbsVariant(Problem):
#     """
#     State: x value for the abs function variant f(x)
#     Move: a new x value delta steps from the current x (in both directions)
#     """

#     def __init__(self, initial, maximum=30.0, delta=0.001):
#         self.initial = initial
#         self.maximum = maximum
#         self.delta = delta

#     def actions(self, state):
#         return [state + self.delta, state - self.delta]

#     def result(self, stateIgnored, x):
#         return x

#     def value(self, x):
#         return math.fabs(x * math.sin(x))
#         # Thought about limiting the problem size but decided against it
#         # if 0 <= x <= 30:
#         #     return math.fabs(x * math.sin(x))
#         # else:
#        #     return 0


if __name__ == '__main__':

    # Formulate a problem with a 2D hill function and a single maximum value.
    citySize = 10
    cityConfig = []
    for i in range(citySize):
        print('\n')
        for j in range(citySize):
            print(i + ' : ' + j)

    # For loop to do find solutions to random starting variables
    # for count in range(restartTimes):
    #     initial = randrange(0, maximum)
    #     time1 = time.time()
    #     p = AbsVariant(initial, maximum, delta=1)
    #     time2 = time.time()
    #     totalValInit += p.value(initial)
    #     timeForInit += (time2 - time1)
    #     if maxValInit < p.value(initial):
    #         maxValInit = p.value(initial)

    #     # Solve the problem using hill-climbing.
    #     time1 = time.time()
    #     hill_solution = hill_climbing(p)
    #     time2 = time.time()
    #     totalValHill += p.value(hill_solution)
    #     timeForHill += (time2 - time1)
    #     if maxValHill < p.value(hill_solution):
    #         maxValHill = p.value(hill_solution)

    #     # Solve the problem using simulated annealing.
    #     time1 = time.time()
    #     annealing_solution = simulated_annealing(
    #         p,
    #         exp_schedule(k=20, lam=0.005, limit=1000)
    #     )
    #     time2 = time.time()
    #     totalValAnneal += p.value(annealing_solution)
    #     timeForAnneal += (time2 - time1)
    #     if maxValAnneal < p.value(annealing_solution):
    #         maxValAnneal = p.value(annealing_solution)

    # # Print all the max values, average values, and times
    # print('Initial: \t\t\t\t\t max value: %0.3f' % maxValInit
    #       + '\t\taverage value: %0.3f' % (totalValInit / restartTimes)
    #       + "\t\ttime: %0.3f seconds" % timeForInit
    #       )
    # print('Hill-climbing solution: \t max value: %0.3f' % (maxValHill)
    #       + '\t\taverage value: %0.3f' % (totalValHill / restartTimes)
    #       + "\t\ttime: %0.3f seconds" % (timeForHill)
    #       )
    # print('Simulated annealing solution max value: %0.3f' % maxValAnneal
    #       + '\t\taverage value: %0.3f' % (totalValAnneal / restartTimes)
    #       + "\t\ttime: %0.3f seconds" % (timeForAnneal)
    #       )


TypeError: unsupported operand type(s) for +: 'int' and 'str'